In [1]:
from pyspark.sql.functions import lit, from_json, col
from pyspark.sql.types import *

In [2]:
display(dbutils.fs.ls("s3araw://###########/json_msgs.parquet"))

In [3]:
msgDF = spark.read.parquet("s3araw://###########/json_msgs/batch/json_msgs.parquet")
display(msgDF)

In [4]:
struct =  StructType([
  StructField("QuoteNumber", StringType(), True),
  StructField("BrowserType", StringType(), False),
  StructField("SenderIpAddress", StringType(), False),
  StructField("RequestUrl", StringType(), False),
  StructField("RedirectUrl", StringType(), False),
  StructField("utm_source", StringType(), False),
  StructField("utm_term", StringType(), False),
  StructField("utm_campaign", StringType(), False),
  StructField("utm_medium", StringType(), False),
  StructField("utm_content", StringType(), False),
  StructField("UtmQueryString", StringType(), False),
  StructField("ClickDate", StringType(), False)])

In [5]:
cleansedMsgDF = msgDF.withColumn("exploded", from_json(msgDF.value, struct)).select("exploded.*")
display(cleansedMsgDF)

In [6]:
cleansedMsgDF.write.saveAsTable("masked.clickRateQuotes", mode='overwrite')

In [7]:
clicks = spark.sql("""

SELECT
DISTINCT(QuoteNumber)
FROM masked.clickRateQoutes

UNION 

SELECT
DISTINCT(QuoteNumber)
FROM masked.clickratequotes_trueup

""")

In [8]:
clicks = clicks.distinct()

In [9]:
deduped_quotes = spark.sql("""

SELECT 
QUOTE_NUMBER 
FROM 
masked.booked_lines_test
WHERE TRIM(OPPORTUNITY_QUOTE_STATUS) IN ("VALID", "ORDER_COMPLETE", "CANCELLED") 
and (BOOKED_QUOTE_NUMBER in (select distinct(QuoteNumber) from clickRateQoutes) 
  or QUOTE_NUMBER in (select distinct(QuoteNumber) from clickRateQoutes) 
  or ORIGINAL_AUTO_QUOTE_NUM in (select distinct(QuoteNumber) from clickRateQoutes))



UNION

SELECT 
QUOTE_NUMBER 
FROM 
masked.bookings_snapshot_table
WHERE TRIM(OPPORTUNITY_QUOTE_STATUS) IN ("VALID", "ORDER_COMPLETE", "CANCELLED") 
and (BOOKED_QUOTE_NUMBER in (select distinct(QuoteNumber) from clickRateQoutes) 
  or QUOTE_NUMBER in (select distinct(QuoteNumber) from clickRateQoutes) 
  or ORIGINAL_AUTO_QUOTE_NUM in (select distinct(QuoteNumber) from clickRateQoutes))



UNION 

SELECT 
QUOTE_NUMBER
FROM
masked.quoted_lines_test
WHERE TRIM(OPPORTUNITY_QUOTE_STATUS) IN ("VALID", "ORDER_COMPLETE", "CANCELLED") 
and (BOOKED_QUOTE_NUMBER in (select distinct(QuoteNumber) from clickRateQoutes) 
  or QUOTE_NUMBER in (select distinct(QuoteNumber) from clickRateQoutes) 
  or ORIGINAL_AUTO_QUOTE_NUM in (select distinct(QuoteNumber) from clickRateQoutes)) 

""")

In [10]:
deduped_quotes = deduped_quotes.withColumn('clicked', lit(1)) \
.withColumnRenamed('QUOTE_NUMBER', 'deduped_QUOTE_NUMBER') \
.distinct() \

display(deduped_quotes)


In [11]:
threshold_df  = spark.sql("""select * from masked.bookings_reporting""")

display(threshold_df)

In [12]:
final_table  = threshold_df.join(deduped_quotes, threshold_df.QUOTE_NUMBER == deduped_quotes.deduped_QUOTE_NUMBER,'left') \
.drop('deduped_QUOTE_NUMBER')

In [13]:
display(final_table)

In [14]:
final_table.write.saveAsTable("masked.clicked_bookings_report", mode='overwrite')